# LIANA

In [38]:
# import liana
import liana as li
# needed for visualization and toy data
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad
from scipy import sparse
import cell2cell

In [13]:
pwd

'/root/workdir/HDAC_tfm/scripts/liana-py'

In [23]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
#X = pd.read_csv("/root/workdir/HDAC_tfm/data/anndata/logcounts.csv", index_col=0)
#obs = pd.read_csv("/root/workdir/HDAC_tfm/data/anndata/obs.csv", index_col=0)
#var = pd.read_csv("/root/workdir/HDAC_tfm/data/anndata/var.csv", index_col=0)

# Transponer X si tiene forma (genes x células) → debe ser (células x genes)
#if X.shape[0] == var.shape[0] and X.shape[1] == obs.shape[0]:
    #X = X.T

# Convertir a matriz dispersa (opcional, pero recomendable para ahorrar memoria)
#X_sparse = sparse.csr_matrix(X.values)

# Crear el objeto AnnData
#adata = ad.AnnData(X=X_sparse, obs=obs, var=var)

# Guardar en formato .h5ad
#adata.write("/root/workdir/HDAC_tfm/data/anndata/sce_object_human.h5ad")

In [17]:
adata = sc.read_h5ad("/root/workdir/HDAC_tfm/data/anndata/sce_object_human.h5ad")

In [18]:
adata

AnnData object with n_obs × n_vars = 72650 × 16652
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'bc1_wind', 'bc2_wind', 'bc3_wind', 'bc1_well', 'bc2_well', 'bc3_well', 'percent.mt', 'percent.rb', 'percent.hb', 'integrated_snn_res.0.095', 'seurat_clusters', 'integrated_snn_res.0.09', 'immuno', 'ident'
    var: 'Unnamed: 0'

In [19]:
li.method.show_methods()

,Method Name,Magnitude Score,Specificity Score,Reference
0,CellPhoneDB,lr_means,cellphone_pvals,"Efremova, M., Vento-Tormo, M., Teichmann, S.A...."
0,Connectome,expr_prod,scaled_weight,"Raredon, M.S.B., Yang, J., Garritano, J., Wang..."
0,log2FC,None,lr_logfc,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,NATMI,expr_prod,spec_weight,"Hou, R., Denisenko, E., Ong, H.T., Ramilowski,..."
0,SingleCellSignalR,lrscore,None,"Cabello-Aguilar, S., Alame, M., Kon-Sun-Tack, ..."
0,Rank_Aggregate,magnitude_rank,specificity_rank,"Dimitrov, D., Türei, D., Garrido-Rodriguez, M...."
0,Geometric Mean,lr_gmeans,gmean_pvals,CellPhoneDBv2's permutation approach applied t...
0,scSeqComm,inter_score,None,"Baruzzo, G., Cesaro, G., Di Camillo, B. 2022. ..."
0,CellChat,lr_probs,cellchat_pvals,"Jin, S., Guerrero-Juarez, C.F., Zhang, L., Cha..."


I will use consensus of these ones!! Important not to use log2FC because for Tensor-cell2cell there can not be negative values!!

In [20]:
immuno_values = adata.obs['immuno']

print(immuno_values.unique())

['Cancer cells', 'Endothelial', 'Natural killer  cells', 'Macrophages', 'Pro-B cells', 'Naive CD8+ T cells', 'Plasmacytoid Dendritic cells', 'Pre-B cells']
Categories (8, object): ['Cancer cells', 'Endothelial', 'Macrophages', 'Naive CD8+ T cells', 'Natural killer  cells', 'Plasmacytoid Dendritic cells', 'Pre-B cells', 'Pro-B cells']


In [24]:
li.mt.rank_aggregate.by_sample(adata,
                               sample_key='sample',
                               groupby='immuno',
                               resource_name = 'consensus',
                               expr_prop=0.1, # must be expressed in expr_prop fraction of cells
                               min_cells = 5,
                               n_perms = 100,
                               use_raw = False, # run on log- and library-normalized counts
                               verbose = True,
                               inplace = True,
                               return_all_lrs=True, # return all LR values
                              )

Now running: KAP25L_control: 100%|████████████████████████████████████████████| 8/8 [02:23<00:00, 17.95s/it]


In [25]:
adata.uns['liana_res'].head()

,sample,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,specificity_rank,magnitude_rank
0,KAP25L-UV_Combination,Macrophages,Naive CD8+ T cells,MRC1,PTPRC,2.974149,0.0,8.830788,2.457145,4.214756,0.459048,0.939541,1.724807e-08,5.778168e-10
1,KAP25L-UV_Combination,Natural killer cells,Pre-B cells,SPON1,APP,2.897716,0.0,8.384596,1.481714,2.660393,0.147084,0.938052,4.239494e-06,2.311255e-09
2,KAP25L-UV_Combination,Pro-B cells,Plasmacytoid Dendritic cells,FGF13,FGFR2,2.751034,0.0,7.539707,2.804802,2.542878,0.089459,0.934893,4.239494e-06,9.244920e-09
3,KAP25L-UV_Combination,Pro-B cells,Pre-B cells,FGF13,FGFR1,2.678331,0.0,7.115149,3.259270,2.886946,0.177293,0.933106,3.850075e-06,1.444511e-08
4,KAP25L-UV_Combination,Macrophages,Macrophages,MRC1,PTPRC,2.656271,0.0,6.862673,2.107425,4.239426,0.356740,0.931970,9.870821e-08,2.080085e-08


In [29]:
adata.uns['liana_res'].to_csv('/root/workdir/HDAC_tfm/data/anndata/LIANA_by_sample.csv', index=False)

In [32]:
sorted_samples = sorted(adata.obs['sample'].unique())

In [35]:
adata.uns['liana_res'].head()

,sample,source,target,ligand_complex,receptor_complex,lr_means,cellphone_pvals,expr_prod,scaled_weight,lr_logfc,spec_weight,lrscore,specificity_rank,magnitude_rank
0,KAP25L-UV_Combination,Macrophages,Naive CD8+ T cells,MRC1,PTPRC,2.974149,0.0,8.830788,2.457145,4.214756,0.459048,0.939541,1.724807e-08,5.778168e-10
1,KAP25L-UV_Combination,Natural killer cells,Pre-B cells,SPON1,APP,2.897716,0.0,8.384596,1.481714,2.660393,0.147084,0.938052,4.239494e-06,2.311255e-09
2,KAP25L-UV_Combination,Pro-B cells,Plasmacytoid Dendritic cells,FGF13,FGFR2,2.751034,0.0,7.539707,2.804802,2.542878,0.089459,0.934893,4.239494e-06,9.244920e-09
3,KAP25L-UV_Combination,Pro-B cells,Pre-B cells,FGF13,FGFR1,2.678331,0.0,7.115149,3.259270,2.886946,0.177293,0.933106,3.850075e-06,1.444511e-08
4,KAP25L-UV_Combination,Macrophages,Macrophages,MRC1,PTPRC,2.656271,0.0,6.862673,2.107425,4.239426,0.356740,0.931970,9.870821e-08,2.080085e-08


In [40]:
adata.obs[['cell_line', 'treat']] = adata.obs['sample'].str.split('_', expand=True)

In [41]:
adata.write_h5ad('/root/workdir/HDAC_tfm/data/anndata/homan_adata_processed_27_05.h5ad')

... storing 'cell_line' as categorical
... storing 'treat' as categorical


In [39]:
# build the tensor
tensor = li.multi.to_tensor_c2c(liana_res=adata.uns['liana_res'], # LIANA's dataframe containing results
                                sample_key='sample', # Column name of the samples
                                source_key='source', # Column name of the sender cells
                                target_key='target', # Column name of the receiver cells
                                ligand_key='ligand_complex', # Column name of the ligands
                                receptor_key='receptor_complex', # Column name of the receptors
                                score_key='magnitude_rank', # Column name of the communication scores to use
                                inverse_fun=lambda x: 1 - x, # Transformation function
                                how='outer_cells', # What to include across all samples
                                outer_fraction=1/3, # Fraction of samples as threshold to include cells and LR pairs.
                                context_order=sorted_samples, # Order to store the contexts in the tensor
                               )

100%|█████████████████████████████████████████████████████████████████████████| 8/8 [05:12<00:00, 39.07s/it]


In [42]:
context_dict = adata.obs.sort_values(by='sample') \
                        .set_index('sample')['cell_line'] \
                        .to_dict()

In [44]:
dimensions_dict = [context_dict, None, None, None]
meta_tensor = cell2cell.tensor.generate_tensor_metadata(interaction_tensor=tensor,
                                                  metadata_dicts=dimensions_dict,
                                                  fill_with_order_elements=True
                                                 )

In [45]:
cell2cell.io.export_variable_with_pickle(variable=tensor,
                                   filename='/root/workdir/HDAC_tfm/data/anndata/Tensor.pkl')
cell2cell.io.export_variable_with_pickle(variable=meta_tensor,
                                   filename='/root/workdir/HDAC_tfm/data/anndata/Tensor-Metadata.pkl')

/root/workdir/HDAC_tfm/data/anndata/Tensor.pkl  was correctly saved.
/root/workdir/HDAC_tfm/data/anndata/Tensor-Metadata.pkl  was correctly saved.


In [46]:
mdata = li.multi.lrs_to_views(adata,
                              sample_key='sample',
                              score_key='magnitude_rank',
                              obs_keys=['treat', 'cell_line'], # add those to mdata.obs
                              lr_prop = 0.3, # minimum required proportion of samples to keep an LR
                              lrs_per_sample = 20, # minimum number of interactions to keep a sample in a specific view
                              lrs_per_view = 20, # minimum number of interactions to keep a view
                              samples_per_view = 5, # NOTE: minimum number of samples to keep a view
                              min_variance = 0, # minimum variance to keep an interaction
                              lr_fill = 0, # fill missing LR values across samples with this
                              verbose=True
                              )

100%|███████████████████████████████████████████████████████████████████████| 64/64 [00:29<00:00,  2.16it/s]


In [47]:
import mudata

In [48]:
mdata.write("/root/workdir/HDAC_tfm/data/anndata/Mudata_human_27_05.h5mu")

In [27]:
plot = li.pl.dotplot_by_sample(adata=adata,
                               colour='magnitude_rank',
                               size='specificity_rank',
                               source_labels = c["Cancer cells"],
                               sample_key='sample',
                               inverse_colour=True,
                               inverse_size=True,
                               figure_size=(12, 9),
                               size_range=(1, 6),
                               )